# Landing Zone
Fitxer que permet obtenir les bases de dades en la seva última versió.

Requeriments previs:

In [1]:
# !pip install chardet
# !pip install pyarrow
# !pip install kaggle

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.4/199.4 kB 31.9 kB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.2/24.2 MB 330.8 kB/s eta 0:00:0000:0100:04

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip


Llibreries necessàries:

In [1]:
import pandas as pd
import requests
from io import StringIO
import chardet

## Descàrrega API Compravenda

ATENCIÓ: la següent base de dades no és pública, la sol·licitud no tindrà èxit.

In [3]:
# sol·licitar el fitxer
response = requests.get('https://opendata-ajuntament.barcelona.cat/data/es/dataset/est-mercat-immobiliari-compravenda-sup/resource/56b5a20f-078f-40f3-a637-bfa455d0ce1d/download')

In [4]:
if response.status_code == 200:
    # detecció del contingut del fitxer
    encoding = chardet.detect(response.content)['encoding']
    
    # descodificació del contingut del fitxer
    contenido = response.content.decode(encoding)
    
    # conversió a un fitxer llegible per pandas i lectura de la base de dades
    csv_file = StringIO(contenido)
    df = pd.read_csv(csv_file)

else:
    print("La solicitud no fue exitosa. Código de estado:", response.status_code)


In [5]:
# guardar la base de dades en un fitxer parquet
#df.to_parquet('data_lake/compravenda_sup.parquet')

## Descàrrega API Renda Tributària per llar

In [2]:
# sol·licitar els fitxers
path = ['https://opendata-ajuntament.barcelona.cat/data/dataset/c9b9b5e5-b02f-4b47-892b-79acf0191802/resource/44916a85-462d-4c1c-b3a5-98d9e161c299/download','https://opendata-ajuntament.barcelona.cat/data/dataset/c9b9b5e5-b02f-4b47-892b-79acf0191802/resource/cb629032-8de5-4a52-ba0a-c5e2410d0911/download', 'https://opendata-ajuntament.barcelona.cat/data/dataset/c9b9b5e5-b02f-4b47-892b-79acf0191802/resource/0d9df8cf-8b95-43c3-b07d-14e11f2ebdaf/download','https://opendata-ajuntament.barcelona.cat/data/dataset/c9b9b5e5-b02f-4b47-892b-79acf0191802/resource/9d30d9b1-bbbe-4431-84f1-bcbf001acc14/download','https://opendata-ajuntament.barcelona.cat/data/dataset/c9b9b5e5-b02f-4b47-892b-79acf0191802/resource/47dabf0e-371d-4739-9206-c81a56666797/download','https://opendata-ajuntament.barcelona.cat/data/dataset/c9b9b5e5-b02f-4b47-892b-79acf0191802/resource/37147963-5686-4746-8b13-9a1349906351/download','https://opendata-ajuntament.barcelona.cat/data/dataset/c9b9b5e5-b02f-4b47-892b-79acf0191802/resource/94d5b7c5-b702-420f-957d-14ce10e3add5/download']

In [3]:
renta_global = pd.DataFrame()

In [4]:
# obtenció de les dades i creació de la base de dades
for i in path:
    response = requests.get(i)
    if response.status_code == 200:
        # detecció del contingut del fitxer
        encoding = chardet.detect(response.content)['encoding']
        
        # descodificació del contingut del fitxer
        contenido = response.content.decode(encoding)
        
        # conversió a un fitxer llegible per pandas i lectura de la base de dades
        csv_file = StringIO(contenido)
        df = pd.read_csv(csv_file)

        # correcció d'un error en el nom d'una columna en algunes dades
        if 'Import_Euros ' in df.columns:
            df = df.rename(columns={'Import_Euros ': 'Import_Euros'})

        # concatenació de les dades
        renta_global = pd.concat([renta_global, df], ignore_index=True)

    else:
        print("La solicitud no fue exitosa. Código de estado:", response.status_code)

In [5]:
# guardar la base de dades en un fitxer parquet
renta_global.to_parquet('data_lake/renda.parquet')

## Descàrrega API preu del lloguer (Kaggle)

Abans s'ha de:
1. Crear un compte a Kaggle
2. Anar a Settings i fer click a Create New API Token
3. Guardar el fitxer kaggle.json que s'acaba de descarregar a la carpeta ~/.kaggle

In [14]:
# requeriments per a evitar problemes de la seguretat de connexió
# import ssl
# ssl._create_default_https_context = ssl._create_unverified_context

In [18]:
# descarregar la base de dades
!kaggle datasets download -d marshuu/rent-price-in-barcelona-2014-2022

/Users/ruthparajo/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
100%|██████████████████████████████████████| 33.6k/33.6k [00:00<00:00, 90.4kB/s]
100%|██████████████████████████████████████| 33.6k/33.6k [00:00<00:00, 90.1kB/s]


In [19]:
# descomprimir la base de dades
!unzip rent-price-in-barcelona-2014-2022.zip

Archive:  rent-price-in-barcelona-2014-2022.zip
  inflating: Barcelona_rent_price.csv  


In [20]:
# guardar la base de dades en un fitxer parquet
pd.read_csv('Barcelona_rent_price.csv').to_parquet('data_lake/rent_price.parquet')

In [21]:
# eliminar els fitxers innecessaris
!rm rent-price-in-barcelona-2014-2022.zip Barcelona_rent_price.csv